# API connection - Auxiliary Functions

In [ ]:
!pip install krakenex

In [3]:
import krakenex
import datetime
import calendar
import pandas as pd
import time
import numpy as np

In [4]:
import bokeh
from bokeh.plotting import figure, output_file, show

In [9]:
#Open the API
k = krakenex.API()

#takes date and returns nix time
def date_nix(str_date):
    try:
        return calendar.timegm(datetime.datetime.strptime(str_date,'%m-%d-%Y %H-%M-%S').date().timetuple())
    except:
        try:
            return calendar.timegm(datetime.datetime.strptime(str_date,'%m-%d-%Y').date().timetuple())
        except:
            print('Wrong date format: format should be "%m-%d-%Y %H-%M-%S" or "%m-%d-%Y" ')
        
#takes nix time and returns date
def date_str(nix_time):
    return datetime.datetime.fromtimestamp(nix_time).strftime('%m-%d-%Y %H-%M-%S')

#return formated request data 
def req_Trades(pair, since=None):
    # !! Need to find the correspondance since id - time
    if since !=None:
        return {'pair': pair, 'since': str(date_nix(since))+"000000000"} 
    else:
        return  {'pair': pair}

#Return a dataframe with the address "function" of the API
def get_data(function,pair, since=None): 
    r= req_Trades(pair, since)
    x=k.query_public(function,r)
    data=x['result'][pair] 
    last_id=x['result']['last']
    return {'df':pd.DataFrame(data), 'last_id':last_id}
    

def get_mean_change(data,serie):
    data[serie]=data[serie].apply(lambda x: float(x))
    mean=np.mean(data[serie])
    change=(data[serie].iloc[len(data[serie])-1]-data[serie].iloc[0])/data[serie].iloc[0]
    Max = np.max(data[serie])
    Min = np.min(data[serie])
    return {'Mean':mean, 'Change':change, 'Max':Max, 'Min':Min}

# DataFrame - Charts - Analytics

In [10]:
#Spread
def get_spread(pair,t=10,since=None):
    data=get_data('Spread',pair,since)
    df=data['df']
    df.columns=['Time','Bid','Ask']
    df['Spread']=df['Bid'].apply(lambda x: float(x))-df['Ask'].apply(lambda x: float(x))
    x=calendar.timegm(time.gmtime())
    #x=int(1+int(data['last_id'])/1000000000)
    x0=x-t*60  
    df=df[df['Time']>=x0]
    bid_dict=get_mean_change(df,'Bid')
    ask_dict=get_mean_change(df,'Ask')
    spread_dict=get_mean_change(df,'Spread')
    return {'Bid':bid_dict, 'Ask':ask_dict, 'Spread':spread_dict, 'df':df}
              
#Recent Trades

def get_Trades(pair,t, since=None):
    df=get_data('Trades',pair,since)['df']
    df.columns=['Price','Volume','Time','Order_type','Market/lim','Misc']
    x=calendar.timegm(time.gmtime())
    x0=x-t*60  
    df=df[df['Time']>=x0]
    price_dict=get_mean_change(df,'Price')
    Volume_dict=get_mean_change(df,'Volume')
    df_buy=df[df['Order_type']=='b']
    df_sell=df[df['Order_type']=='s']
    price_buy_dict=get_mean_change(df_buy,'Price')
    Volume_buy_dict=get_mean_change(df_buy,'Volume')
    price_sell_dict=get_mean_change(df_sell,'Price')
    Volume_sell_dict=get_mean_change(df_sell,'Volume')
    
    #Graph
    
    #Change time format
    df['Time']=df['Time'].apply(lambda x: datetime.datetime.fromtimestamp(int(x)))
    df_buy['Time']=df_buy['Time'].apply(lambda x: datetime.datetime.fromtimestamp(int(x)))
    df_sell['Time']=df_sell['Time'].apply(lambda x: datetime.datetime.fromtimestamp(int(x)))
    #Outputfile                                      
    output_file("Trades_graph.html") 
    #add graph                                      
    p = figure(title="Price over periode", x_axis_label='x', y_axis_label='y',width=800, height=500,x_axis_type="datetime")                            
    p.line(df_sell['Time'], df_sell['Price'], legend="XXBTZEUR_sell", line_width=2)
    p.line(df_buy['Time'], df_buy['Price'], legend="XXBTZEUR_buy", line_width=2,line_color="red")
    p.circle(df['Time'], df['Price'], fill_color="white",line_color='black', size=8)
                                                                            
    return {'Price':price_dict,'Volume':Volume_dict,'df':df,'Graph':p,'Buy_Order':{'Price':price_buy_dict, 'Volume':Volume_buy_dict},'Sell_Order':{'Price':price_sell_dict, 'Volume':Volume_sell_dict}}

# Flask WebApp


In [ ]:
!pip install flask_wtf

In [ ]:
import flask
from flask import Flask, session, render_template,url_for,request
from flask_wtf import Form
from flask import Flask, session, render_template,url_for,request
from flask_wtf import Form
from wtforms import SelectField

In [ ]:
app=Flask(__name__)
app.secret_key = 'A0Zr98slkjdf984jnflskj_sdkfjhT'

def get_homepage_links():
    return [ {"href": url_for('Function'), "label":"Select Function"},]

class UserForm(Form):
    user = SelectField('Select Function', choices=list('Trades'))

@app.route('/')
def home():
    session['data_loaded']= True
    return render_template('home.html',links=get_homepage_links())

@app.route('/Function')
def Function():
    print("Here will be the functions")

if __name__ == "__main__":
    app.run()    

In [ ]:
app.close()

In [ ]:
calendar.timegm(datetime.datetime.strptime(14900000087,'%m-%d-%Y %H-%M-%S').date().timetuple()

In [ ]:
datetime.datetime.now()

In [13]:
get_data('Trades','XXBTZEUR')

{'df':               0           1             2  3  4 5 
 0    2553.73000  3.35475000  1.497124e+09  b  l   
 1    2553.74400  0.10000000  1.497124e+09  b  l   
 2    2553.98000  0.04099836  1.497124e+09  b  l   
 3    2553.99900  0.10000000  1.497124e+09  b  l   
 4    2554.00000  2.16177311  1.497124e+09  b  l   
 5    2553.20000  0.14000000  1.497124e+09  s  l   
 6    2553.00000  0.00177542  1.497124e+09  s  l   
 7    2552.98800  0.36122959  1.497124e+09  s  l   
 8    2552.98800  0.13877041  1.497124e+09  s  l   
 9    2552.98800  0.00702959  1.497124e+09  b  m   
 10   2553.70000  0.10000000  1.497124e+09  b  l   
 11   2554.00000  0.26000000  1.497124e+09  b  l   
 12   2553.01300  0.03229820  1.497124e+09  s  l   
 13   2552.98800  0.00496192  1.497124e+09  b  l   
 14   2554.00000  0.64000000  1.497124e+09  b  l   
 15   2552.73300  0.93342093  1.497124e+09  s  l   
 16   2554.00000  0.94513914  1.497124e+09  b  m   
 17   2552.73300  0.10180000  1.497124e+09  s  l   
 18   

In [11]:
pd.DataFrame(get_data('Trades','XXBTZEUR')['df'])[2].astype(int).head()# since=None

0    1497123889
1    1497123890
2    1497123892
3    1497123898
4    1497123926
Name: 2, dtype: int64

In [16]:
calendar.timegm(datetime.datetime.strptime('09-06-2017 15-00-00','%m-%d-%Y %H-%M-%S').date().timetuple()

SyntaxError: unexpected EOF while parsing (<ipython-input-16-bbc9af1ae357>, line 1)

In [19]:
strptime('09-06-2017 15-00-00','%m-%d-%Y %H-%M-%S')

NameError: name 'strptime' is not defined

In [18]:
calendar.timegm(datetime.datetime.strptime('09-06-2017 15-00-00','%m-%d-%Y %H-%M-%S').date().timetuple())

1504656000

In [20]:
datetime.datetime.strptime('09-06-2017 15-00-00','%m-%d-%Y %H-%M-%S')

datetime.datetime(2017, 9, 6, 15, 0)